In [34]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Connection details for source and target databases
server_source = 'MENNAS-LAPTOP\\MSSQLSERVER22'
database_source = 'AdventureWorks2019'
username_source = 'sa'
password_source = 'sa123456'
driver = 'ODBC Driver 17 for SQL Server'

server_target = 'MENNAS-LAPTOP\\MSSQLSERVER22'
database_target = 'HR'
username_target = 'sa'
password_target = 'sa123456'



# Create engine for source and target databases
connection_string_source = (
    f'mssql+pyodbc://{username_source}:{password_source}@{server_source}/{database_source}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_source = create_engine(connection_string_source, echo=False)

connection_string_target = (
    f'mssql+pyodbc://{username_target}:{password_target}@{server_target}/{database_target}'
    f'?driver={driver.replace(" ", "+")}'
)
engine_target = create_engine(connection_string_target, echo=False)


In [ ]:

import traceback
import logging
from datetime import date,timedelta,datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
logger = logging.getLogger(__name__)

import getpass


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail_error(
    sender="osamamenna1511@gmail.com",  # put sender email here 
    password='lwrx jrtd uhjm cjvt',  # add a comma here
    to_list=["menna.elminshawy@yahoo.com"],  # receiver list here
    error=None,
    subject="Error AD",
):
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = ", ".join(to_list)
    msg["Subject"] = subject
    message = f"""<html><body>{error}</body></html>"""
    msg.attach(MIMEText(message, "html"))
    
    # sends email
    smtpserver = smtplib.SMTP("smtp.gmail.com", 587)
    smtpserver.starttls()
    smtpserver.login(sender, password)  # use the 'sender' variable instead of hardcoding
    smtpserver.sendmail(msg["From"], to_list, msg.as_string())
    smtpserver.quit()


## UNIQUNESS CHECK


In [35]:
Session = sessionmaker(bind=engine_target)
def get_tables(schema_name):
    """
    Retrieve all tables in the specified schema.

    :param schema_name: The schema name to retrieve tables from.
    :return: A list of table names in the specified schema.
    """
    query = f"""
    SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = :schema_name
    """

    try:
        with Session() as session:
            with session.begin():
                # Execute the query with the provided schema name
                result = session.execute(text(query), {'schema_name': schema_name})
                # Fetch all results and extract table names
                tables = [row['TABLE_NAME'] for row in result.mappings()]
        
        print(f"Tables in schema '{schema_name}':")
        print(tables)
        return tables
    
    except Exception as e:
        print(f"Error retrieving tables from schema '{schema_name}': {e}")
        return []

In [36]:

def get_columns(schema_name, table_name):
    """
    Retrieve all columns for the specified table in the schema, excluding XML columns.

    :param schema_name: The schema name where the table resides.
    :param table_name: The name of the table to retrieve columns from.
    :return: A list of column names in the specified table, excluding XML columns.
    """
    query = f"""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = :schema_name
      AND TABLE_NAME = :table_name
      AND DATA_TYPE <> 'xml'
    """

    try:
        with Session() as session:
            with session.begin():
                # Execute the query with the provided schema and table names
                result = session.execute(text(query), {'schema_name': schema_name, 'table_name': table_name})
                # Fetch all results and extract column names
                columns = [row['COLUMN_NAME'] for row in result.mappings()]
        
        print(f"Columns in table '{schema_name}.{table_name}':")
        print(columns)
        return columns
    
    except Exception as e:
        print(f"Error retrieving columns from table '{schema_name}.{table_name}': {e}")
        return []


In [37]:
def check_uniqueness(schema_name, table_name, unique_columns):
    """
    Check for uniqueness in the specified table based on the unique columns.
    
    :param schema_name: The schema name where the table resides.
    :param table_name: The name of the table to check.
    :param unique_columns: A list of columns that should have unique values.
    """
    if not unique_columns:
        print(f"No columns to check for uniqueness in {schema_name}.{table_name}.")
        return

    unique_columns_str = ', '.join(unique_columns)
    
    query = f"""
    SELECT {unique_columns_str}, COUNT(*) as count
    FROM {schema_name}.{table_name}
    GROUP BY {unique_columns_str}
    HAVING COUNT(*) > 1
    """
    
    try:
        with Session() as session:
            with session.begin():
                # Execute the query and load the result into a DataFrame
                result = session.execute(text(query))
                df_duplicates = pd.DataFrame(result.fetchall(), columns=result.keys())

        if not df_duplicates.empty:
            print(f"Duplicate records found in {schema_name}.{table_name} based on columns {unique_columns_str}:")
            print(df_duplicates)
        else:
            print(f"No duplicates found in {schema_name}.{table_name} based on columns {unique_columns_str}.")
    
    except Exception as e:
        print(f"Error checking uniqueness in {schema_name}.{table_name}: {e}")


In [38]:
schema_name = 'dimension'

# Get all tables from the schema
tables = get_tables(schema_name)

# Loop through each table and check columns
for table in tables:
    columns = get_columns(schema_name, table)
    if columns:
        check_uniqueness(schema_name, table, columns)


Tables in schema 'dimension':
['Employeelogin', 'Department', 'Shift', 'EmployeeDepartmentHistory', 'Employee_Payment', 'Personal_information']
Columns in table 'dimension.Employeelogin':
['BusinessEntityID', 'LoginID', 'rowguid']
No duplicates found in dimension.Employeelogin based on columns BusinessEntityID, LoginID, rowguid.
Columns in table 'dimension.Department':
['DepartmentID', 'Name', 'GroupName', 'ModifiedDate']
No duplicates found in dimension.Department based on columns DepartmentID, Name, GroupName, ModifiedDate.
Columns in table 'dimension.Shift':
['ShiftID', 'Name', 'StartTime', 'EndTime', 'ModifiedDate']
No duplicates found in dimension.Shift based on columns ShiftID, Name, StartTime, EndTime, ModifiedDate.
Columns in table 'dimension.EmployeeDepartmentHistory':
['EmployeeID', 'BusinessEntityID', 'ShiftID', 'StartDate', 'EndDate', 'ModifiedDate', 'JobTitle', 'IsCurrent', 'Timestamp', 'scd_startdate', 'scd_enddate', 'ALT_shiftid']
No duplicates found in dimension.Employe

## Nulls Check

In [39]:
def check_nulls(schema_name, table_name):
    columns = get_columns(schema_name, table_name)
    if not columns:
        print(f"No columns found for table '{schema_name}.{table_name}'.")
        return
    
    null_counts = []
    
    for column in columns:
        query = f"""
        SELECT COUNT(*) as null_count
        FROM {schema_name}.{table_name}
        WHERE {column} IS NULL
        """
        
        try:
            with Session() as session:
                with session.begin():
                    result = session.execute(text(query))
                    null_count = result.scalar()  # Fetch the count of NULLs
                    null_counts.append((column, null_count))
        
        except Exception as e:
            print(f"Error checking NULLs in column '{column}' of table '{schema_name}.{table_name}': {e}")
    
    # Print results
    print(f"NULL values in table '{schema_name}.{table_name}':")
    for column, count in null_counts:
        print(f"Column '{column}': {count} NULL values")



In [40]:
# Example usage
schema_name = 'dimension'

# Get all tables from the schema
tables = get_tables(schema_name)

# Loop through each table and check for NULLs in columns
for table in tables:
    check_nulls(schema_name, table)

Tables in schema 'dimension':
['Employeelogin', 'Department', 'Shift', 'EmployeeDepartmentHistory', 'Employee_Payment', 'Personal_information']
Columns in table 'dimension.Employeelogin':
['BusinessEntityID', 'LoginID', 'rowguid']
NULL values in table 'dimension.Employeelogin':
Column 'BusinessEntityID': 0 NULL values
Column 'LoginID': 0 NULL values
Column 'rowguid': 0 NULL values
Columns in table 'dimension.Department':
['DepartmentID', 'Name', 'GroupName', 'ModifiedDate']
NULL values in table 'dimension.Department':
Column 'DepartmentID': 0 NULL values
Column 'Name': 0 NULL values
Column 'GroupName': 0 NULL values
Column 'ModifiedDate': 0 NULL values
Columns in table 'dimension.Shift':
['ShiftID', 'Name', 'StartTime', 'EndTime', 'ModifiedDate']
NULL values in table 'dimension.Shift':
Column 'ShiftID': 0 NULL values
Column 'Name': 0 NULL values
Column 'StartTime': 0 NULL values
Column 'EndTime': 0 NULL values
Column 'ModifiedDate': 0 NULL values
Columns in table 'dimension.EmployeeDep

## SALARY SUM CHECK Accuracy

In [42]:
SourceSession = sessionmaker(bind=engine_source)

def get_sum_from_source(source_schema, source_table, column_name):
    query = f"""
    SELECT SUM({column_name}) AS total_sum
    FROM {source_schema}.{source_table}
    """
    
    try:
        with SourceSession() as session:
            result = session.execute(text(query))
            total_sum = result.scalar()
        
        print(f"Sum of '{column_name}' in source table '{source_schema}.{source_table}': {total_sum}")
        return total_sum
    
    except Exception as e:
        print(f"Error retrieving sum from source table '{source_schema}.{source_table}': {e}")
        return None

In [43]:
TargetSession = sessionmaker(bind=engine_target)
def get_sum_from_target(target_schema, target_table, target_column_name):
    query = f"""
    SELECT SUM({target_column_name}) AS total_sum
    FROM {target_schema}.{target_table}
    """
    
    try:
        with TargetSession() as session:
            result = session.execute(text(query))
            total_sum = result.scalar()
        
        print(f"Sum of '{target_column_name}' in target table '{target_schema}.{target_table}': {total_sum}")
        return total_sum
    
    except Exception as e:
        print(f"Error retrieving sum from target table '{target_schema}.{target_table}': {e}")
        return None


In [44]:
def compare_sums(source_schema, source_table, target_schema, target_table, target_column_name):
    
    source_sum = get_sum_from_source(source_schema, source_table, column_name)
    target_sum = get_sum_from_target(target_schema, target_table, target_column_name)
    
    if source_sum is not None and target_sum is not None:
        if source_sum == target_sum:
            print(f"Sums are equal for column '{column_name}': {source_sum}")
        else:
            print(f"Discrepancy found. Source sum: {source_sum}, Target sum: {target_sum}")

In [45]:
# Example usage
source_schema = 'HumanResources'
source_table = 'EmployeePayHistory'
column_name = 'Rate'  # Source column
target_schema = 'dimension'
target_table = 'Employee_Payment'
target_column_name = 'Salary_hourly_rate'  # Target column

compare_sums(source_schema, source_table, target_schema, target_table, target_column_name)

Sum of 'Rate' in source table 'HumanResources.EmployeePayHistory': 5611.7821
Sum of 'Salary_hourly_rate' in target table 'dimension.Employee_Payment': 5611.7821
Sums are equal for column 'Rate': 5611.7821


## National ID check validity


In [46]:
Session = sessionmaker(bind=engine_target)

def percentage_invalid_national_ids(schema_name, table_name, column_name):
    # Query to count invalid National ID numbers
    invalid_count_query = f"""
    SELECT COUNT(*) AS invalid_count
    FROM {schema_name}.{table_name}
    WHERE LEN({column_name}) != 9 OR ISNUMERIC({column_name}) = 0
    """
    
    # Query to count total entries in the table
    total_count_query = f"""
    SELECT COUNT(*) AS total_count
    FROM {schema_name}.{table_name}
    """
    
    try:
        with Session() as session:
            # Get the count of invalid entries
            invalid_result = session.execute(text(invalid_count_query))
            invalid_count = invalid_result.scalar()
            
            # Get the total count of entries
            total_result = session.execute(text(total_count_query))
            total_count = total_result.scalar()
        
        if total_count == 0:
            print("The table is empty, no entries to evaluate.")
            return 0
        
        # Calculate the percentage of invalid entries
        percentage_invalid = (invalid_count / total_count) * 100
        print(f"Percentage of invalid {column_name} entries in {schema_name}.{table_name}: {percentage_invalid:.2f}%")
        return percentage_invalid
    
    except Exception as e:
        print(f"Error calculating percentage of invalid {column_name} entries: {e}")
        return None

# Example usage
schema_name = 'dimension'
table_name = 'Personal_information'
column_name = 'NationalIDNumber'

percentage_invalid_national_ids(schema_name, table_name, column_name)


Percentage of invalid NationalIDNumber entries in dimension.Personal_information: 13.79%


13.793103448275861

In [47]:
def check_national_id_summary(schema_name, table_name, column_name, primary_key):
    query = f"""
    SELECT {primary_key} AS row_id, {column_name} AS value_with_issue
    FROM {schema_name}.{table_name}
    WHERE LEN({column_name}) != 9 OR ISNUMERIC({column_name}) = 0
    """
    
    try:
        with Session() as session:
            result = session.execute(text(query))
            rows = result.fetchall()
        
        if rows:
            print(f"Issues found in {schema_name}.{table_name}:")
            print(f"{'row_id':<10}{'DQ_rule':<20}{'Action':<10}{'Table':<25}{'Column with issue':<20}{'table Primary key':<20}{'table Primary key':<20}{'Value with issue':<30}")
            for row in rows:
                print(f"{row.row_id:<10}{'National ID check':<20}{'Review':<10}{table_name:<25}{column_name:<20}{primary_key:<20}{row.row_id:<20}{row.value_with_issue:<30}")
        else:
            print(f"No issues found in {schema_name}.{table_name} for column {column_name}.")
    
    except Exception as e:
        print(f"Error checking {column_name} in {schema_name}.{table_name}: {e}")

# Example usage
schema_name = 'dimension'
table_name = 'Personal_information'
column_name = 'NationalIDNumber'
primary_key = 'BusinessEntityID'

check_national_id_summary(schema_name, table_name, column_name, primary_key)

Issues found in dimension.Personal_information:
row_id    DQ_rule             Action    Table                    Column with issue   table Primary key   table Primary key   Value with issue              
14        National ID check   Review    Personal_information     NationalIDNumber    BusinessEntityID    14                  42487730                      
15        National ID check   Review    Personal_information     NationalIDNumber    BusinessEntityID    15                  56920285                      
16        National ID check   Review    Personal_information     NationalIDNumber    BusinessEntityID    16                  24756624                      
19        National ID check   Review    Personal_information     NationalIDNumber    BusinessEntityID    19                  52541318                      
22        National ID check   Review    Personal_information     NationalIDNumber    BusinessEntityID    22                  95958330                      
24        Nation

In [48]:
import csv
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Assume engine_target is already created
Session = sessionmaker(bind=engine_target)

def check_national_id_summary(schema_name, table_name, column_name, primary_key, output_file='national_id_issues.csv'):
    query = f"""
    SELECT {primary_key} AS row_id, {column_name} AS value_with_issue
    FROM {schema_name}.{table_name}
    WHERE LEN({column_name}) != 9 OR ISNUMERIC({column_name}) = 0
    """
    
    try:
        with Session() as session:
            result = session.execute(text(query))
            rows = result.fetchall()
        
        if rows:
            # Write results to a CSV file
            with open(output_file, mode='w', newline='') as file:
                writer = csv.writer(file)
                # Write header
                writer.writerow(['row_id', 'DQ_rule', 'Action', 'Table', 'Column with issue', 'table Primary key','table Primary key value' 'Value with issue'])
                
                # Write data rows
                for row in rows:
                    writer.writerow([
                        row.row_id,
                        'National ID check',
                        'Review',
                        table_name,
                        column_name,
                        primary_key,
                        row.value_with_issue
                    ])
            
            print(f"Issues found in {schema_name}.{table_name} and written to {output_file}")
        else:
            print(f"No issues found in {schema_name}.{table_name} for column {column_name}.")
    
    except Exception as e:
        print(f"Error checking {column_name} in {schema_name}.{table_name}: {e}")

# Example usage
schema_name = 'dimension'
table_name = 'Personal_information'
column_name = 'NationalIDNumber'
primary_key = 'BusinessEntityID'

check_national_id_summary(schema_name, table_name, column_name, primary_key)


Issues found in dimension.Personal_information and written to national_id_issues.csv


## Timeliness

In [49]:
from datetime import datetime, timedelta
def check_timeliness(schema_name, table_name, timestamp_column, threshold_hours=24):
    query = f"""
    SELECT MAX({timestamp_column}) AS latest_update
    FROM {schema_name}.{table_name}
    """
    
    try:
        with Session() as session:
            result = session.execute(text(query))
            latest_update = result.scalar()
        
        if latest_update:
            latest_update = datetime.strptime(str(latest_update), '%Y-%m-%d %H:%M:%S.%f')
            current_time = datetime.now()
            time_difference = current_time - latest_update
            
            if time_difference <= timedelta(hours=threshold_hours):
                print(f"Data in {schema_name}.{table_name} is up-to-date.")
            else:
                print(f"Data in {schema_name}.{table_name} is NOT up-to-date. Last update was {time_difference.total_seconds() / 3600:.2f} hours ago.")
        else:
            print(f"No update timestamp found in {schema_name}.{table_name}.")
    
    except Exception as e:
        print(f"Error checking timeliness in {schema_name}.{table_name}: {e}")

# Example usage
schema_name = 'dimension'
table_name = 'EmployeeDepartmentHistory'
timestamp_column = 'LoadTimestamp'  # Replace with your actual timestamp column

check_timeliness(schema_name, table_name, timestamp_column)

Error checking timeliness in dimension.EmployeeDepartmentHistory: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'LoadTimestamp'. (207) (SQLExecDirectW)")
[SQL: 
    SELECT MAX(LoadTimestamp) AS latest_update
    FROM dimension.EmployeeDepartmentHistory
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [50]:
from sqlalchemy import create_engine, MetaData, Table, select
import pandas as pd

def check_duplicates_national_id(engine):
    """Check for duplicate NationalIDNumber in dimension.Personal_information."""
    with engine.connect() as connection:
        # Load the metadata and table
        metadata = MetaData()
        PersonalInformation = Table('Personal_information', metadata, autoload_with=engine, schema='dimension')

        # Query to fetch all data from the table
        query = select(PersonalInformation.c.NationalIDNumber)
        results = connection.execute(query).fetchall()

        # Convert to DataFrame for easier manipulation
        df = pd.DataFrame(results, columns=['NationalIDNumber'])

        # Check for duplicates
        duplicates = df[df.duplicated('NationalIDNumber', keep=False)]

        return duplicates

# Example usage
duplicates_df = check_duplicates_national_id(engine_target)
print("Duplicate NationalIDNumbers:")
print(duplicates_df)

C:\Users\menna\AppData\Local\Temp\ipykernel_30692\3865131173.py:9: SAWarning: Did not recognize type 'hierarchyid' of column 'employee_location_in_corporate_hierarchy'
  PersonalInformation = Table('Personal_information', metadata, autoload_with=engine, schema='dimension')


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near ')'. (102) (SQLExecDirectW)")
[SQL: WITH fk_info AS (
    SELECT
        ischema_ref_con.constraint_schema,
        ischema_ref_con.constraint_name,
        ischema_key_col.ordinal_position,
        ischema_key_col.table_schema,
        ischema_key_col.table_name,
        ischema_ref_con.unique_constraint_schema,
        ischema_ref_con.unique_constraint_name,
        ischema_ref_con.match_option,
        ischema_ref_con.update_rule,
        ischema_ref_con.delete_rule,
        ischema_key_col.column_name AS constrained_column
    FROM
        INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS ischema_ref_con
        INNER JOIN
        INFORMATION_SCHEMA.KEY_COLUMN_USAGE ischema_key_col ON
            ischema_key_col.table_schema = ischema_ref_con.constraint_schema
            AND ischema_key_col.constraint_name =
            ischema_ref_con.constraint_name

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, select
import pandas as pd

def check_negative_salary(engine):
    """Check if Salary_hourly_rate is non-negative in dimension.Employee_Payment."""
    # Create a connection to the database
    with engine.connect() as connection:
        # Load the metadata and table
        metadata = MetaData()
        EmployeePayment = Table('Employee_Payment', metadata, autoload_with=engine, schema='dimension')

        # Query to find records with negative Salary_hourly_rate
        query = select(EmployeePayment).where(EmployeePayment.c.Salary_hourly_rate < 0)
        results = connection.execute(query).fetchall()

        # Convert to DataFrame for better readability (optional)
        df = pd.DataFrame(results, columns=results[0].keys() if results else [])
    
    return df

# Example usage
negative_salaries_df = check_negative_salary(engine_target)
print(negative_salaries_df)

Empty DataFrame
Columns: []
Index: []


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

def extract_and_validate_data(schema_name, table_name, column_names, validation_func, *args):
    query = f"""
    SELECT {', '.join(column_names)}
    FROM [{schema_name}].[{table_name}]
    """
    
    try:
        # Extract data into a DataFrame
        df = pd.read_sql(query, engine_target)
        print(f"Data from {schema_name}.{table_name} loaded into DataFrame:")
        print(df.head())

        # Apply the validation function
        _, invalid_data = validation_func(df, *args)

        # Print only invalid entries
        if invalid_data is not None and not invalid_data.empty:
            print(f"Invalid entries:")
            print(invalid_data.head())
        else:
            print("No invalid entries found.")
        
        return invalid_data

    except Exception as e:
        print(f"Error extracting or validating data: {e}")
        return None

# Define your validation function to check StartDate < EndDate
def validate_dates_comparison(df, start_date_col, end_date_col):
    # Convert columns to datetime, forcing errors to NaT
    df[start_date_col] = pd.to_datetime(df[start_date_col], errors='coerce')
    df[end_date_col] = pd.to_datetime(df[end_date_col], errors='coerce')
    
    # Invalid entries: StartDate >= EndDate and EndDate is not NaT
    invalid_data = df[~((df[start_date_col] <= df[end_date_col]) | (df[end_date_col].isna()))]
    
    return None, invalid_data

# Example usage
schema_name = 'dimension'  # Correct schema name
table_name = 'EmployeeDepartmentHistory'  # Correct table name
column_names = ['StartDate', 'EndDate']  # List columns you need to select

extract_and_validate_data(schema_name, table_name, column_names, validate_dates_comparison, 'StartDate', 'EndDate')

Data from dimension.EmployeeDepartmentHistory loaded into DataFrame:
    StartDate     EndDate
0  2009-01-14  2024-08-17
1  2024-08-12  2024-08-17
2  2024-08-12  2024-08-17
3  2008-01-31        None
4  2020-08-12        None
No invalid entries found.


,StartDate,EndDate
